# Create map of a stream reach based on a specified NWIS site and range, including various feature layers derived from USGS NLDI navigation along the stream, its tributaries and associated drainage basins, while also extracting data for further analysis (plotting and statistics)

<h2>Quick index:</h2>
<ul>
    <li><a href=#reach>Define River Reach</a></li>
    <li><a href=#map>Go to Map</a></li>
    <li><a href=#gage_selection>USGS Gage Selection for Plots</a></li>
    <li><a href=#gage_selection>USGS Gage Plots</a></li>
</ul>

### TODO:  
* Buid lists for
 * NWIS stations
 * WQP stations
 * HUC12 PPs (and HUC12 shapes)
 * HUC12 PP drainage basins
 * HUC12 PPs up tribs (and HUC12 shapes) -- how to best find these
 * Counties -- later
 * NPDES sources -- when available again
 * NOAA/NWS weather stations -- after providing a way to enter API key and search for nearby stations

* Put data (and/or URIs) in database
* Query NWIS Sites by clicking on a map and specifying a range
* How to locate HUC12s up tribs (not main stream) that share same PP ???
* Locate NOAA weather stations within HUC (needs API key, need way to provide)
* Generate interactive plots after map
 * Try Seaborn or Bokeh (in addition to Pandas and Matplotlib)
   * Time series for USGS gage flow and stage
   * Property map for WQP properties (see Greenup/ORSANCO)
 * Try Scipy.stats, Statsmodels or SKLearn for statistical modeling
   * See  Box & Whiskey plots for Greenup

## Get things set up

#### To install any packages locally (e.g., folium, geopandas, fiona):
```
%%bash -l
use -e -r anaconda3-5.1
pip install folium --prefix=. --upgrade-strategy only-if-needed
```

#### To access them:
```
import os, sys
sys.path.insert(0,'lib/python3.7/site-packages') # Add locally installed packages to path
```

In [267]:
import os.path
from os import path

### Use full width of browser window

In [268]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Enable all variable displays within a cell
 * Terminate statement with semicolon or assign to a variable ("X =") to suppress output

In [269]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Enable numerical arrays and matrices with NumPy

In [270]:
import numpy as np

### Enable R-style DataFrames with Pandas

In [271]:
import pandas as pd

### Enable Math functions

In [272]:
import math

### Enable working with Dates and Times

In [273]:
import time
from datetime import datetime

### Enable inline MATLAB-style plotting with MatPlotLib

In [274]:
import matplotlib.pyplot as plt
%matplotlib inline

### Make plots (even matplotlib plots) prettier with Seaborn

In [275]:
import seaborn as sbn

### Enable interactive functions (especially plots) using iPyWidgets

In [276]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

### Enable geospatial DataFrames with GeoPandas (built on Fiona, which is built on GDAL/OGR)

In [277]:
import geopandas as gpd

### Enable other geospatial functions using Shapely

In [278]:
import shapely
from shapely.geometry import Point, Polygon

### Enable Leatlet.JS-based mapping with Folium 

In [279]:
import folium
from folium import IFrame
import folium.plugins as plugins

### Enable additional HTML features with Branca

In [280]:
import branca

### Enable HTTP requests and parsing of JSON results

In [281]:
import requests
import json

### Enable SQL database access using SQLAlchemy (and GeoAlchemy)

In [282]:
from sqlalchemy import create_engine  
from sqlalchemy import Table, Column, Integer, String, MetaData  
from sqlalchemy import select, func
from sqlalchemy.ext.declarative import declarative_base  
from sqlalchemy.event import listen
from sqlalchemy.orm import sessionmaker, relationship, backref
from geoalchemy2 import Geometry

### Define some handy geometry functions

In [283]:
def geom_distance(lat1, lon1, lat2, lon2):
    R = 6378.137 # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d # Km

In [284]:
def geom_diagonal(geom):
    lon1 = geom.total_bounds[0]
    lat1 = geom.total_bounds[1]
    lon2 = geom.total_bounds[2]
    lat2 = geom.total_bounds[3]
    d = geom_distance(lat1, lon1, lat2, lon2)
    return d # Km

In [285]:
def geom_extent(geom):
    d2 = geom_width(geom)+geom_height(geom)
    return d2

In [286]:
def geom_height(geom):
    lon1 = geom.total_bounds[0]
    lat1 = geom.total_bounds[1]
    lon2 = geom.total_bounds[2]
    lat2 = geom.total_bounds[3]
    h = geom_distance(lat1, lon1, lat2, lon1)
    return h # Km

In [287]:
def geom_width(geom):
    lon1 = geom.total_bounds[0]
    lat1 = geom.total_bounds[1]
    lon2 = geom.total_bounds[2]
    lat2 = geom.total_bounds[3]
    w = geom_distance(lat1, lon1, lat1, lon2)
    return w # Km

In [288]:
def geom_bbox(geom):
    polygon = gpd.GeoDataFrame(gpd.GeoSeries(geom.envelope), columns=['geometry'])
    return polygon

In [289]:
# In case CRS is different

def geom_bbox2(geom):
    lon_point_list = [geom.total_bounds[0],geom.total_bounds[2],geom.total_bounds[2],geom.total_bounds[0],geom.total_bounds[0]]
    lat_point_list = [geom.total_bounds[1],geom.total_bounds[1],geom.total_bounds[3],geom.total_bounds[3],geom.total_bounds[1]]
    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    crs = {'init': 'epsg:4326'}
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])    
    return polygon

In [290]:
# Try using the area of the total_bounds polygon in both degrees and meters to generate an approximate "conversion" factor

def geom_area(geom):
    factor = geom_width(geom)*geom_height(geom)/geom_bbox(geom).area
    area = factor*geom.area
    return area # Km^2

In [291]:
# Use a cartesian projection coordinate system to get true area
# *** Currently crashes kernel ***

def geom_area2(geom):
    geom_m = geom.to_crs(epsg=3857) # or epsg=32633
    a = geom_m.area/10**6
    return a # Km^2

<div id='reach' />

### Define the river reach by setting the starting location (NWIS Station) and up/down-stream distance ranges (Km)

In [292]:
from IPython.display import Javascript
def run_all_below(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
    
run_button = widgets.Button(description=" Rerun all cells below ") 
run_button.on_click(run_all_below)

reach = widgets.VBox(
    [
        widgets.Label(
            value='Select NWIS Station and range (Km):\n'
        ),
        widgets.Dropdown(
            options=[
                ('Huntington','USGS-03206000'), # <== Add other NWIS stations here
                ('Charleston','USGS-03198000')
            ], 
            value='USGS-03206000', # <== Default goes here
            description='NWIS Site:'
        ),
        widgets.IntSlider(
            value=25,
            min=0,
            max=100,
            step=5,
            description='Upstream:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        ),
        widgets.IntSlider(
            value=25,
            min=0,
            max=100,
            step=5,
            description='Downstream:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        ),
        run_button
    ]
)
display(reach)

In [293]:
NWIS_SITE = reach.children[1].value
UM_DIST = reach.children[2].value
DM_DIST = reach.children[3].value

In [294]:
print("Reach ==> NWIS Station: {0} + {1} Km upstream and {2} Km downstream".format(NWIS_SITE, UM_DIST, DM_DIST))

Reach ==> NWIS Station: USGS-03206000 + 25 Km upstream and 25 Km downstream


### URLs for REST Web Services

In [295]:
USGS_NLDI_WS = "https://labs.waterdata.usgs.gov/api/nldi/linked-data" # USGS NLDI REST web services
NWIS_SITE_URL = USGS_NLDI_WS+"/nwissite/"+NWIS_SITE 
NWIS_SITE_NAV = NWIS_SITE_URL+"/navigate"
TNM_WS = "https://hydro.nationalmap.gov/arcgis/rest/services" # The National Map REST web services
ARCGIS_WS = "http://server.arcgisonline.com/arcgis/rest/services" # ARCGIS Online REST web services

### Set Input (optional) and Output Directories
  * TODO: Need option to upload files to Input Directory

In [296]:
IN_DIR = "data/"+NWIS_SITE
OUT_DIR = IN_DIR+"/out"

### Create output directory if it does not already exist

In [297]:
%mkdir -p {OUT_DIR}

### Get Lat/Lon coordinates of starting site (NWIS station)

In [298]:
nwis_site_json = gpd.read_file(NWIS_SITE_URL)
nwis_site_geom = nwis_site_json.iloc[0]['geometry']
nwis_site_coord = [nwis_site_geom.y, nwis_site_geom.x]

### Set up a local file-based SQLite/Spatialite database

In [299]:
spatialite_db_filename = 'spatialite.db'
spatialite_engine = create_engine('sqlite:///{0}/{1}'.format(OUT_DIR,spatialite_db_filename))

In [300]:
def load_spatialite(dbapi_conn, connection_record):
    dbapi_conn.enable_load_extension(True)
    dbapi_conn.load_extension('/opt/conda/lib/mod_spatialite.so')
    
listen(spatialite_engine, 'connect', load_spatialite)

In [301]:
spatialite_engine._metadata = MetaData(bind=spatialite_engine)
spatialite_engine._metadata.reflect(spatialite_engine)

In [302]:
spatialite_conn = spatialite_engine.connect()

In [303]:
spatialite_conn.execute(select([func.InitSpatialMetaData()]))

In [304]:
Base = declarative_base()

class Site(Base):
    __tablename__ = 'site'
    id = Column(Integer, primary_key=True)
    type = Column(String)
    name = Column(String)
    desc = Column(String)
    url = Column(String)
    geom = Column(Geometry(geometry_type='POINT',management=True))

Site.__table__.create(spatialite_engine, checkfirst=True)

In [305]:
#spatialite_engine._metadata.sorted_tables

In [306]:
Spatialite_Session = sessionmaker(bind=spatialite_engine)
spatialite_session = Spatialite_Session()

In [307]:
anchor_site_in = Site(type='ANCHOR',name=NWIS_SITE,desc='Anchor site for reach',url=NWIS_SITE_URL,geom='POINT(('+str(nwis_site_geom.y)+' '+str(nwis_site_geom.x)+'))')
spatialite_session.add(anchor_site_in)
spatialite_session.commit()

In [308]:
anchor_site_out = spatialite_session.query(Site).filter_by(type='ANCHOR')

for row in anchor_site_out:
    print(row.name)

USGS-03206000
USGS-03206000


## Generate map

In [309]:
river_map = folium.Map(nwis_site_coord,zoom_start=10,tiles=None)
plugins.ScrollZoomToggler().add_to(river_map);
plugins.Fullscreen(
    position='bottomright',
    title='Full Screen',
    title_cancel='Exit Full Screen',
    force_separate_button=True
).add_to(river_map);

### Add NWIS and WQP sites within reach using NLDI web services at USGS

In [310]:
# Popup parameters

width = 500
height = 120
max_width = 1000

# Main Stream

folium.GeoJson(NWIS_SITE_NAV+"/UM?distance="+str(UM_DIST),name="Main Stream (up)",show=True,control=False).add_to(river_map);
folium.GeoJson(NWIS_SITE_NAV+"/DM?distance="+str(DM_DIST),name="Main Stream (down)",show=True,control=False).add_to(river_map);

# NWIS Sites

fg_nwis = folium.FeatureGroup(name="USGS (NWIS) Sites",overlay=True,show=False)
color = 'darkred'
icon = 'dashboard'
        
nwis_sites_dm = gpd.read_file(NWIS_SITE_NAV+"/DM/nwissite?distance="+str(DM_DIST))
nwis_sites_um = gpd.read_file(NWIS_SITE_NAV+"/UM/nwissite?distance="+str(UM_DIST))
nwis_sites = gpd.GeoDataFrame(pd.concat([nwis_sites_dm,nwis_sites_um], ignore_index=True), crs=nwis_sites_dm.crs)

for i, nwis_site in nwis_sites.iterrows():
    coord = [nwis_site.geometry.y,nwis_site.geometry.x]
    label = 'NWIS Station: '+nwis_site.identifier+" ("+str(i)+")"
    html = label
    html += '<br>{0:s}'.format(nwis_site['name'])
    html += '<br><a href=\"{0:s}\",target=\"_blank\">{1:s}'.format(nwis_site.uri,nwis_site.uri)+'</a>'
    html += '<br>Lat: {0:.2f}'.format(nwis_site.geometry.y)+', Lon: {0:.2f}'.format(nwis_site.geometry.x)
    html += '<br>Comid: {0:s}'.format(nwis_site.comid)
    iframe = folium.IFrame(html,width=width,height=height)
    popup = folium.Popup(iframe,max_width=max_width)
    fg_nwis.add_child(folium.Marker(location=coord,icon=folium.Icon(color=color,icon=icon),popup=popup,tooltip=label));
    nwis_site = Site(type='NWIS',name=nwis_site.identifier,desc=nwis_site.name,url=nwis_site.uri,geom='POINT(('+str(nwis_site.geometry.y)+' '+str(nwis_site.geometry.x)+'))')
    spatialite_session.add(nwis_site)
    
spatialite_session.commit()
        
fg_nwis.add_to(river_map)

# WQP Stations
        
fg_wqp = folium.FeatureGroup(name="WQP Stations",overlay=True,show=False)
color = 'darkgreen'
radius = 3
        
wqp_sites_dm = gpd.read_file(NWIS_SITE_NAV+"/DM/wqp?distance="+str(DM_DIST))
wqp_sites_um = gpd.read_file(NWIS_SITE_NAV+"/UM/wqp?distance="+str(UM_DIST))
wqp_sites = gpd.GeoDataFrame(pd.concat([wqp_sites_dm,wqp_sites_um], ignore_index=True), crs=wqp_sites_dm.crs)

for i, wqp_site in wqp_sites.iterrows():
    coord = [wqp_site.geometry.y,wqp_site.geometry.x]
    label = 'WQP Station: '+wqp_site.identifier+" ("+str(i)+")"
    html = label
    html += '<br>{0:s}'.format(wqp_site['name'])
    html += '<br><a href=\"{0:s}\",target=\"_blank\">{1:s}'.format(wqp_site.uri,wqp_site.uri)+'</a>'
    html += '<br>Lat: {0:.2f}'.format(wqp_site.geometry.y)+', Lon: {0:.2f}'.format(wqp_site.geometry.x)
    html += '<br>Comid: {0:s}'.format(wqp_site.comid)
    iframe = folium.IFrame(html,width=width,height=height)
    popup = folium.Popup(iframe,max_width=max_width)
    fg_wqp.add_child(folium.CircleMarker(location=coord,radius=radius,color=color,popup=popup,tooltip=label));
    wqp_site = Site(type='WQP',name=wqp_site.identifier,desc=wqp_site.name,url=wqp_site.uri,geom='POINT(('+str(wqp_site.geometry.y)+' '+str(wqp_site.geometry.x)+'))')
    spatialite_session.add(wqp_site)

spatialite_session.commit()

fg_wqp.add_to(river_map);

### Add HUC12 Pour Points and *differential* drainage basins associated with each

In [311]:
fg_huc12pp = folium.FeatureGroup(name="HUC12 Pour Points",overlay=True,show=False)
fg_basins = folium.FeatureGroup(name="Drainage Basins",overlay=True,show=False)

color = 'darkblue'
radius = 3
        
try:
    huc12pp_sites_dm = gpd.read_file(NWIS_SITE_NAV+"/DM/huc12pp?distance="+str(DM_DIST),driver='GeoJSON')
except Exception as ex:
    print("An exception of type {0} occurred. Arguments:\n{1!r}".format(type(ex).__name__, ex.args))
    huc12pp_sites_dm = gpd.GeoDataFrame()
    
try:
    huc12pp_sites_um = gpd.read_file(NWIS_SITE_NAV+"/UM/huc12pp?distance="+str(UM_DIST),driver='GeoJSON')
except Exception as ex:
    print("An exception of type {0} occurred. Arguments:\n{1!r}".format(type(ex).__name__, ex.args))
    huc12pp_sites_um = gpd.GeoDataFrame()
    
huc12pp_sites = gpd.GeoDataFrame(pd.concat([huc12pp_sites_dm,huc12pp_sites_um], ignore_index=True), crs=huc12pp_sites_dm.crs)

n_segs = len(huc12pp_sites)-1

# Sort sites by decreasing area of drainage basin
def get_area(x):
    x_basin = gpd.read_file(USGS_NLDI_WS+"/comid/"+x+"/basin")
    return int(round(x_basin.iloc[0].geometry.area,3)*1000)  

huc12pp_sites['area']=huc12pp_sites.apply(lambda x: get_area(x.comid), axis=1)
huc12pp_sites.set_index(['area'],inplace=True,drop=True)
huc12pp_sites.sort_index(inplace=True,ascending=False)

i = 0
i_max = n_segs
huc12pp_comid_list = []
huc12_list = []
huc10_list = []

for area, huc12pp_site in huc12pp_sites.iterrows():
    basin_url = USGS_NLDI_WS+"/comid/{0:s}/basin".format(huc12pp_site.comid)
    
    try:
        basin = gpd.read_file(basin_url,driver='GeoJSON')
    except Exception as ex:
        print("An exception of type {0} occurred. Arguments:\n{1!r}".format(type(ex).__name__, ex.args))
        i = i + 1
        continue
        
    #basin_area = basin.iloc[0].geometry.area
    basin_area = geom_area(basin)
    basin_diff_area = basin_area
    wbd12_url = TNM_WS+"/wbd/MapServer/6/query?where=HUC12%3D%27{0:s}%27&outFields=NAME%2CHUC12%2CSHAPE_Length&f=geojson".format(huc12pp_site.identifier)
    
    try:
        wbd12 = gpd.read_file(wbd12_url,driver='GeoJSON')
    except Exception as ex:
        print("An exception of type {0} occurred. Arguments:\n{1!r}".format(type(ex).__name__, ex.args))
        i = i + 1
        continue

        if (huc12pp_site.identifier not in huc12_list):
            huc12_list.append(huc12pp_site.identifier)
        if (huc12pp_site.comid not in huc12pp_comid_list):
            huc12pp_comid_list.append(huc12pp_site.comid)
            
        huc10_identifier = huc12pp_site.identifier[:-2]
            
        if (huc10_identifier in huc10_list):
            continue
        else:
            huc10_list.append(huc10_identifier)

    if i > 0:
        # Generate and show difference (polygon) between sussessive drainage basins
        basin_diff = gpd.overlay(basin_prev,basin,how='difference')  
        basin_diff_area = geom_area(basin_diff)
        style_function = lambda x: {'color': 'red', 'weight': 1, 'fillColor': 'blue', 'fillOpacity': 0.1}
        highlight_function = lambda x: {'color':'yellow', 'weight':3}
        tooltip = "Differential Drainage Basin for HUC12 Pour Point: {0:s} ({1:s}), Area: {2:.2f}".format(wbd12.iloc[0].HUC12,wbd12.iloc[0].NAME,basin_diff_area[0])
        basin_diff_feature = folium.GeoJson(basin_diff.iloc[0].geometry.buffer(-0.001).buffer(0.001),style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
        fg_basins.add_child(basin_diff_feature);
        
        if i == i_max:
            # Show large basin of first (highest upstream) pour point
            style_function = lambda x: {'color': 'gray', 'weight': 1, 'fillColor': 'gray', 'fillOpacity': 0.1}
            highlight_function = lambda x: {'color':'yellow', 'weight':1}
            tooltip = "Total Drainage Basin for HUC12 Pour Point: {0:s} ({1:s}), Area: {2:.2f}".format(wbd12.iloc[0].HUC12,wbd12.iloc[0].NAME,basin_area[0])
            basin_feature = folium.GeoJson(basin.iloc[0].geometry,style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
            fg_basins.add_child(basin_feature);
            
    basin_prev = basin

    # HUC12 Pour Point markers
    coord = [huc12pp_site.geometry.y,huc12pp_site.geometry.x]
    label = 'Pour Point for HUC12: '+wbd12.iloc[0].NAME
    html = label
    html += '<br>Indentifier: {0:s}'.format(huc12pp_site.identifier)
    html += '<br>Lat: {0:.2f}, Lon: {1:.2f}'.format(huc12pp_site.geometry.y,huc12pp_site.geometry.x)
    html += '<br>Comid: {0:s}'.format(huc12pp_site.comid)
    html += '<br>Area Total: {0:.2f}'.format(basin_area[0])
    html += '<br>Area Difference: {0:.2f}'.format(basin_diff_area[0])
    iframe = folium.IFrame(html,width=width,height=height)
    popup = folium.Popup(iframe,max_width=max_width)
    fg_huc12pp.add_child(folium.CircleMarker(location=coord,radius=radius,color=color,popup=popup,tooltip=label));
    
    i = i + 1

fg_huc12pp.add_to(river_map);
fg_basins.add_to(river_map);

### Add HUC12 and HUC10 WBD boundaries from The National Map web services (ArcGIS) for Pour Points along main stream and up nearby tributaries
 * TODO: Get HUC12s from up-tribs themselves instead the Pour Points

fg_wbd12 = folium.FeatureGroup(name="HUC12 Boundaries",overlay=True,show=False)
fg_wbd12_ut = folium.FeatureGroup(name="Up-Trib HUC12s",overlay=True,show=False)
fg_wbd10 = folium.FeatureGroup(name="HUC10 Boundaries",overlay=True,show=False)
        
i = 0
i_max = n_segs
huc12pp_ut_comid_list = []
huc12_ut_list = []
huc10_ut_list = []

for area, huc12pp_site in huc12pp_sites.iterrows():

    # Get HUC12 watershed boundary (WBD) for pour point from The National Map web services
    wbd12_url = TNM_WS+"/wbd/MapServer/6/query?where=HUC12%3D%27{0:s}%27&outFields=NAME%2CHUC12%2CSHAPE_Length&f=geojson".format(huc12pp_site.identifier)

    try:
        wbd12 = gpd.read_file(wbd12_url)
    except:
        i = i + 1
        continue
        
    if (huc12pp_site.identifier not in huc12_ut_list): # TODO: exclude if in original list, too
        huc12_ut_list.append(huc12pp_site.identifier)
    if (huc12pp_site.comid not in huc12pp_ut_comid_list):
        huc12pp_ut_comid_list.append(huc12pp_site.comid)
    
    if i < i_max:
        style_function = lambda x: {'color': 'darkgreen', 'weight': 1, 'fillColor': 'green', 'fillOpacity': 0.1}
        highlight_function = lambda x: {'color':'yellow', 'weight':2}
        tooltip = "HUC12: {0:s} ({1:s}), Area: {2:.2f}".format(wbd12.iloc[0].HUC12,wbd12.iloc[0].NAME,geom_area(wbd12)[0])
        wbd12_feature = folium.GeoJson(wbd12.iloc[0].geometry,style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
        fg_wbd12.add_child(wbd12_feature);

        # Get extent (width + height) of WBD associated with HUC12 Pour Point
        distance = int(round(geom_extent(wbd12),0))

        # Find other pour points (and associated HUC12s) up stream (following tributares) within the current HUC12
        huc12pp_ut_url = USGS_NLDI_WS+"/comid/{0:s}/navigate/UT/huc12pp?distance={1:d}".format(huc12pp_site.comid,distance)
        huc12pp_ut_sites = gpd.read_file(huc12pp_ut_url)
        
        for k, huc12pp_ut in huc12pp_ut_sites.iterrows():
            if (huc12pp_ut.comid in huc12pp_ut_comid_list):
                continue            
            else:
                huc12pp_ut_comid_list.append(huc12pp_ut.comid)
            
            if (huc12pp_ut.identifier in huc12_ut_list):
                continue                
            else:
                huc12_ut_list.append(huc12pp_ut.identifier)
            
            # Get and display WBD for HUC12 associated with up-trib Pour Point
            wbd12_ut_url = TNM_WS+"/wbd/MapServer/6/query?where=HUC12%3D%27{0:s}%27&outFields=NAME%2CHUC12%2CSHAPE_Length&f=geojson".format(huc12pp_ut.identifier)

            try: 
                wbd12_ut = gpd.read_file(wbd12_ut_url)
            except:
                continue
                
            style_function = lambda x: {'color': 'darkgreen', 'weight': 1, 'fillColor': 'green', 'fillOpacity': 0.05}
            highlight_function = lambda x: {'color':'yellow', 'weight':2}
            tooltip = "HUC12: {0:s} ({1:s}), Area: {2:.2f}".format(wbd12_ut.iloc[0].HUC12,wbd12_ut.iloc[0].NAME,geom_area(wbd12_ut)[0])
            wbd12_ut_feature = folium.GeoJson(wbd12_ut,style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
            fg_wbd12_ut.add_child(wbd12_ut_feature);        

            # Put marker at up-trib HUC12 Pour Point
            coord = [huc12pp_ut.geometry.y,huc12pp_ut.geometry.x]
            label = 'UT Pour Point for HUC12: '+wbd12_ut.iloc[0].NAME
            fg_wbd12_ut.add_child(folium.CircleMarker(location=coord,radius=4,color='lightblue',tooltip=label));

            # Get and display HUC10 containing HUC12
            wbd10_url = TNM_WS+"/wbd/MapServer/5/query?where=HUC10%3D%27{0:s}%27&outFields=NAME%2CHUC10%2CSHAPE_Length&f=geojson".format(huc10_identifier)
            try:
                wbd10 = gpd.read_file(wbd10_url)
            except:
                continue
                
            huc10_identifier = huc12pp_ut.identifier[:-2]
            
            if (huc10_identifier in huc10_ut_list):
                continue
            else:
                huc10_ut_list.append(huc10_identifier)
                                
            style_function = lambda x: {'color': 'darkgreen', 'weight': 1, 'fillColor': 'green', 'fillOpacity': 0.05}
            highlight_function = lambda x: {'color':'yellow', 'weight':2}
            tooltip = "HUC10: {0:s} ({1:s}), Area: {2:.2f}".format(wbd10.iloc[0].HUC10,wbd10.iloc[0].NAME,geom_area(wbd10)[0])
            wbd10_feature = folium.GeoJson(wbd10_url,style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
            fg_wbd10.add_child(wbd10_feature);  
            
    i = i + 1

fg_wbd12.add_to(river_map);
fg_wbd12_ut.add_to(river_map);
fg_wbd10.add_to(river_map);

### Add tributaries upstream of HUC12 Pour Points

In [312]:
fg_utpp = folium.FeatureGroup(name="Tribs upstream of PPs",overlay=True,show=False)

for huc12 in huc12_list:
    wbd12_url = TNM_WS+"/wbd/MapServer/6/query?where=HUC12%3D%27{0:s}%27&f=geojson".format(huc12)
    
    try:
        wbd12 = gpd.read_file(wbd12_url)
    except:
        continue
        
    distance = int(round(geom_diagonal(wbd12),0))
    #distance = 35
    tribs = folium.GeoJson(USGS_NLDI_WS+"/huc12pp/{0:s}/navigate/UT?distance={1:d}".format(huc12,distance))
    fg_utpp.add_child(tribs);
    
fg_utpp.add_to(river_map);

### Add built-in basemaps

In [313]:
folium.TileLayer('StamenTerrain').add_to(river_map);
folium.TileLayer('OpenStreetMap').add_to(river_map);

### Add other basemap options using Map Servers at  ArcGIS Online

In [314]:
mapserver_dict = dict(
    NatGeo_World_Map=ARCGIS_WS+'/NatGeo_World_Map/MapServer',
    World_Street_Map=ARCGIS_WS+'/World_Street_Map/MapServer',
    World_Imagery=ARCGIS_WS+'/World_Imagery/MapServer',
    World_Topo_Map=ARCGIS_WS+'/World_Topo_Map/MapServer',
    World_Shaded_Relief=ARCGIS_WS+'/World_Shaded_Relief/MapServer',
    World_Terrain_Base=ARCGIS_WS+'/World_Terrain_Base/MapServer',
#   World_Physical_Map=ARCGIS_WS+'/World_Physical_Map/MapServer',  # only shows if zoomed out
)
mapserver_query = '/MapServer/tile/{z}/{y}/{x}'

for tile_name, tile_url in mapserver_dict.items():
    tile_url += mapserver_query
    _ = folium.TileLayer(tile_url,name=tile_name,attr=tile_name).add_to(river_map);

### Add the Layer Control widget

In [315]:
folium.LayerControl().add_to(river_map);

### Save the map as HTML

In [316]:
river_map.save(OUT_DIR+'/river_map.html')

<div id='map' />

## Display Map

In [317]:
from IPython.display import IFrame
IFrame(OUT_DIR+"/river_map.html", width=1500, height=750)

## USGS Gage Data

### Some defined constants
 * TODO: Move property limits to dictionary

In [318]:
TURB_MIN = 0.0
TURB_MAX = 400.0
CHLOR_MIN = 0.0
CHLOR_MAX = 12.0
BGA_MIN = 0.05
BGA_MAX = 1.2
SPCOND_MIN = 100.0
SPCOND_MAX = 500.0
DO_MIN = 5.0
DO_MAX = 15.0
PAR_MIN = 0.0
PAR_MAX = 2000.0
NITR_MIN = 0.5
NITR_MAX = 1.5
GAGE_HGT0 = 31.0 # TODO: Need to get this from site elevation
FT_PER_M = 3.28

#### Get NWIS Sites (USGS Gage Stations) from database created for this river reach

In [319]:
nwis_sites = spatialite_session.query(Site).filter_by(type='NWIS')

nwis_sites_menu = []
for row in nwis_sites:
    nwis_sites_menu.append(row.name)

<div id='gage_selection'></div>

### Select Gage and date range

In [326]:
start_date = datetime(2018, 1, 1) # Earliest start date
end_date = datetime.now()
dates = pd.date_range(start_date, end_date, freq='M')
date_options = [(date.strftime(' %b %Y '), date) for date in dates]
date_index = (0, len(date_options)-1)

date_range_slider = widgets.SelectionRangeSlider(
    options=date_options,
    index=date_index,
    description = "Dates",
    orientation = 'horizontal',
    layout = {'width': '500px'}
)

usgs_gage_selector = widgets.VBox(
    [
        widgets.Label(
            value='Select USGS Gage (NWIS Station):\n'
        ),
        widgets.Dropdown(
            options=nwis_sites_menu, 
            value=NWIS_SITE, 
            description='NWIS Station:'
        ),
        date_range_slider,
        run_button
    ]
)
display(usgs_gage_selector)

In [327]:
USGS_gage = usgs_gage_selector.children[1].value
USGS_begin_date = usgs_gage_selector.children[2].value[0]
USGS_end_date = usgs_gage_selector.children[2].value[1]

In [328]:
begin_date_str = "{0}-{1}-{2}".format(USGS_begin_date.year,USGS_begin_date.month,USGS_begin_date.day)
end_date_str = "{0}-{1}-{2}".format(USGS_end_date.year,USGS_end_date.month,USGS_end_date.day)

### Get USGS Gage data directly from USGS (WaterData) web services 
* TODO:
  * Create selector for property(ies)
  * Create dictionary of property codes, names, and order returned:
     *  168408 72254 Water velocity reading from field sensor, feet per second
     *  209867 72255 Mean water velocity for discharge computation, feet per second
     *  209869 00060 Discharge, cubic feet per second
     *  231500 00011 Temperature, water, degrees Fahrenheit, ADVM
     *   59675 00065 Gage height, feet
     *   59676 00010 Temperature, water, degrees Celsius
     *   59677 00400 pH, water, unfiltered, field, standard units
     *   59678 00095 Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius
     *   59679 00300 Dissolved oxygen, water, unfiltered, milligrams per liter
     *   59680 63680 Turbidity, water, unfiltered, monochrome near infra-red LED light, 780-900 nm, detection angle 90 +-2.5 degrees, formazin nephelometric units (FNU)
     *   59681 99133 Nitrate plus nitrite, water, in situ, milligrams per liter as nitrogen
* Varies with gage
* Order out does not match order of params
* See: https://nwis.waterdata.usgs.gov/usa/nwis, https://waterservices.usgs.gov/rest/
* Can use dv instead of uv to get daily summary data (no timezone field)
* Can also receive data in WaterML and JSON formats
* Some USGS gages (e.g., USGS-03206000, Huntington) also collect precipitation data (cb_00045)

In [330]:
usgs_gage = pd.read_csv("https://nwis.waterdata.usgs.gov/usa/nwis/uv/"+ \
                        "?site_no="+USGS_gage[5:]+ \
                        "&period=&begin_date="+begin_date_str+"&end_date="+end_date_str+ \
                        "&cb_00065=on"+ \
                        "&format=rdb", \
                       sep='\t',comment='#',header=[0,1], \
                       dtype={4: np.float64}, \
                       na_values=['Eqp'])

In [331]:
usgs_gage = usgs_gage.reset_index()
usgs_gage.columns = ['Index', 'Agency', 'Site', 'DateTime', 'TZ', 'Gage height', 'qa'] # TODO: Need to lookup property name from a dictionary

In [332]:
usgs_gage = usgs_gage.drop(usgs_gage.columns[6:8:2],axis=1) # TODO: set upper limit based on number of properties requested/returned
usgs_gage = usgs_gage.drop(['Index','Agency','Site','TZ'],axis=1)

#### Limit property ranges
 * TODO: use list properties and dictionary to dynamically select columns and limits

In [333]:
#usgs_gage['Turbidity'] = usgs_gage['Turbidity'].apply(lambda x: TURB_MIN if x < TURB_MIN else x)  # Examples only
#usgs_gage['Turbidity'] = usgs_gage['Turbidity'].apply(lambda x: TURB_MAX if x > TURB_MAX else x)

#### Compute depth (in meters) from Gage height (in feet)

In [334]:
usgs_gage['Depth'] = usgs_gage.apply(lambda x: (x['Gage height']-GAGE_HGT0)/FT_PER_M, axis=1) # TODO: GAGE_HGT0 should be determined from gage elevation (above sea-level)

#### Index by Date-time

In [335]:
def datetime_usgs(row):
    pattern = '%Y-%m-%d %H:%M'
    dt = row['DateTime']
    return pd.to_datetime(time.mktime(time.strptime(dt,pattern)),unit='s')
    
usgs_gage['DateTime'] = usgs_gage.apply(lambda row: datetime_usgs(row),axis=1)
usgs_gage = usgs_gage.set_index(['DateTime'])

#### Put data in database
 * TODO: Need to define property table

<div id='usgs_plots'></div>

## USGS Gage Plots

### Distribution plots

In [336]:
def dist_plot_ts_gage(Property,Kind='kde'):
    usgs_gage[Property].plot(kind=Kind,color='blue')
        
interact(dist_plot_ts_gage, \
         Property=usgs_gage.columns, \
         Kind=['hist','kde','box'] \
        )

interactive(children=(Dropdown(description='Property', options=('Gage height', 'Depth'), value='Gage height'),…

<function __main__.dist_plot_ts_gage(Property, Kind='kde')>

### Correlation plots

In [337]:
def scatter_plot_ts_gage(YProperty=usgs_gage.columns[0],XProperty=usgs_gage.columns[1],CProperty=usgs_gage.columns[1],ColorMap='jet'):
    fig, ax = plt.subplots()
    usgs_gage.plot.scatter(ax=ax,x=XProperty,y=YProperty,c=CProperty,cmap=ColorMap,s=1)
    ax.set_title("Correlation of "+YProperty+" and "+XProperty,fontsize=14)
    
interact(scatter_plot_ts_gage, \
         YProperty=usgs_gage.columns, \
         XProperty=usgs_gage.columns, \
         CProperty=usgs_gage.columns, \
         ColorMap=['jet','viridis','magma','coolwarm','seismic','Greys','Reds','Blues'] \
        )

interactive(children=(Dropdown(description='YProperty', options=('Gage height', 'Depth'), value='Gage height')…

<function __main__.scatter_plot_ts_gage(YProperty='Gage height', XProperty='Depth', CProperty='Depth', ColorMap='jet')>

#### Disable scrolling for time-serie plots

In [338]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

### Time-series plots

In [339]:
def plot_ts_gage(date_range=(start_date,end_date)):
    fig, axes = plt.subplots(nrows=len(usgs_gage.columns),ncols=1,figsize=(20,20)) # TODO: set rows based on number of properties
    i = 0
    for property in usgs_gage.columns:
        usgs_gage[property].plot(ax=axes[i],drawstyle='steps-post',legend=True,color='blue')
        axes[i].legend([property],loc='upper left')
        axes[i].set_xlim(date_range)
        i = i+1
            
interact(plot_ts_gage,date_range=date_range_slider)

interactive(children=(SelectionRangeSlider(description='Dates', index=(0, 27), layout=Layout(width='500px'), o…

<function __main__.plot_ts_gage(date_range=(datetime.datetime(2018, 1, 1, 0, 0), datetime.datetime(2020, 5, 15, 19, 47, 55, 800423)))>

## Water Quality Data

In [340]:
wqp_sites = spatialite_session.query(Site).filter_by(type='WQP')

wqp_sites_menu = []
for row in wqp_sites:
    wqp_sites_menu.append(row.name)

<div id='select_wqp_station'></div>

In [341]:
wqp_station = widgets.VBox(
    [
        widgets.Label(
            value='Select WQP Station:\n'
        ),
        widgets.Dropdown(
            options=wqp_sites_menu, 
            value=wqp_sites_menu[0], 
            description='WQP Station:'
        ),
        date_range_slider,
        run_button
    ]
)
display(wqp_station)